# 5-2. 회귀 모형의 변수 선택

In [2]:
# from sklearn.datasets import load_boston
# boston = load_boston()
# X = boston.data
# y = boston.target

import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

##  데이터 표준화

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
x_test_scaled = scaler.transform(X_test)

## 회귀 계수
LinearRegression 모형의 회귀 계수를 이용해 중요도가 높은 변수들부터 출력

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [6]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

LinearRegression()

In [7]:
cross_val_score(lr, X_train_scaled, y_train, cv=5)

array([0.76048562, 0.74205105, 0.65989017, 0.64044755, 0.70231421])

In [8]:
lr.coef_

array([ -9.4446358 ,   3.43773103,   0.24975748,   2.43132008,
        -8.08735148,  19.97797991,  -0.14366966, -12.21611581,
         5.29130655,  -3.90717391,  -9.80716282,   4.31931654,
       -17.06150049])

In [51]:
import pandas as pd
import numpy as np
feature_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 
                 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
feature_importance_lr = pd.DataFrame(np.c_[np.array(feature_names) ,lr.coef_.ravel()], columns = ['feature', 'coef'])

In [53]:
feature_importance_lr.sort_values("coef", ascending=False, inplace=True)
feature_importance_lr

,feature,coef
8,RAD,5.2913065512034
11,B,4.319316542614849
1,ZN,3.4377310286731353
3,CHAS,2.4313200759178506
5,RM,19.97797991044073
2,INDUS,0.24975747504166287
10,PTRATIO,-9.80716281589696
0,CRIM,-9.444635796580762
4,NOX,-8.087351484918901
9,TAX,-3.907173907195334


## SelectBest 
`SelectKBest` 클래스는 가장 높은 score에 따라 k개의 특징을 선택
> sklearn.feature_selection.SelectKBest(score_func=<function f_classif>, k=10)
- scores_ : 특징들의 점수들
- best_estimator_ : p-valu들, 만일, score_func가 scores만 반환하면 None 

- score_func : callable 타입, 기본값 f_classif, 함수는 두개의 배열 X와 y를 취하고 한쌍의 배열 (scores, pvlaues) 또는 scores가 있는 단일 배열을 반환 합니다.  
              기본 함수는 분류 작업에서만 동작. 아래는 사용할 수 있는 다른 함수
  - f_classif : 분류를 위한 label/feature 사이의 ANOVA F-value입니다.
  - mutual_info_classif : 이산 변수를 위한 상호정보
  - chi2 : 분류를 위한 비 음수의 카이제곱 통계
  - f_regression : 회귀를 위한 label/feature 사이의 F-value
  - mutual_info_regression : 연속형 타겟을 위한 상호정보
  - SelectPercentile : 가장 높은 점수의 백분위 수를 기반으로 특징을 선택

- K : 정수 또는  "all", 기본값 10, 선택할 상위 특징 개수를 지정



In [1]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, chi2
X, y = load_iris(return_X_y=True)
X.shape

(150, 4)

In [2]:
X_new = SelectKBest(chi2, k=1).fit_transform(X, y)
X_new.shape

(150, 1)